# Tensorflow 基础篇

```
这篇博客记录一个Tensorflow程序的基本框架。
```

## 1. 基础概念

理解Tensorflow的第一步要理解两个概念：图（Graph）和对话（Session）。

一个 TensorFlow 的**Graph**描述了计算的过程。

Tensorflow的后端是c++计算模块，和后端的一个连接称为一个**session**。

TensorFlow 程序通常被组织成一个构建阶段和一个执行阶段。在构建阶段, op 的执行步骤被描述成一个图。在执行阶段， 使用会话执行执行图中的 op。

拿神经网络举例：通常在构建阶段创建一个图来表示和训练神经网络，然后在执行阶段反复执行图中的训练 op。

### 1.1 图的相关概念：


**图（tf.Graph）**：代表模型的数据流，由ops和tensor组成， 由tf.get_default_graph()获取;

简单地说：op是操作，是结点；tensor是数据流，是边。

**操作（tf.Operation）**：计算单元，是组成图的结点，操作张量；

**张量（tf.Tensor）**：代表Operation的**符号**输入或输出。Tensor不保存实际的数据而保存得到输出的方法。

构建默认图：

In [1]:
import tensorflow as tf

# 创建一个常量 op, 产生一个 1x2 矩阵. 这个 op 被作为一个节点
# 加到默认图中.
#
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.

matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

上面的图有3个op，包括两个constant() op 和一个 matmul() op。

### 1.2 会话
创建完图以后要启动图：启动图的第一步是创建一个 Session 对象, 如果无任何创建参数, 会话构造器将启动默认图。

In [2]:
# 启动默认图.
sess = tf.Session()

# 调用 sess 的 'run()' 方法来执行矩阵乘法 op, 传入 'product' 作为该方法的参数. 
# 上面提到, 'product' 代表了矩阵乘法 op 的输出, 传入它是向方法表明, 我们希望取回
# 矩阵乘法 op 的输出.
#
# 整个执行过程是自动化的, 会话负责传递 op 所需的全部输入. op 通常是并发执行的.
# 
# 函数调用 'run(product)' 触发了图中三个 op (两个常量 op 和一个矩阵乘法 op) 的执行.
#
# 返回值 'result' 是一个 numpy `ndarray` 对象.
result = sess.run(product)
print result
# ==> [[ 12.]]

# 任务完成, 关闭会话.
sess.close()

[[ 12.]]


指定设备（般你不需要显式指定使用 CPU 还是 GPU, TensorFlow 能自动检测）：

In [3]:
with tf.Session() as sess:
  with tf.device("/gpu:0"):
    matrix1 = tf.constant([[3., 3.]])
    matrix2 = tf.constant([[2.],[2.]])
    product = tf.matmul(matrix1, matrix2)
    result = sess.run([product])
  print(result)

[array([[ 12.]], dtype=float32)]


### 1.3 交互式使用（如在ipython中使用）：

使用 InteractiveSession 代替 Session 类；

使用 Tensor.eval() 和 Operation.run() 方法代替 Session.run()。这样可以避免使用一个变量来持有会话。


In [2]:
# InteractiveSession 相当于建立了一个可交互的长连接。
sess = tf.InteractiveSession()

InternalError: Failed to create session.

In [5]:
x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器 initializer op 的 run() 方法初始化 'x' 
x.initializer.run()

# 增加一个减法 sub op, 从 'x' 减去 'a'. 运行减法 op, 输出结果 
sub = tf.sub(x, a)
print sub.eval()
# ==> [-2. -1.]

[-2. -1.]


### 1.4 Variable

上面的代码还用到了Variable，即变量。变量是带有内存数据缓存的特殊的tensor。

它需要初始化，也可以保存到硬盘。

通常直接初始化所有变量：

```python
# init_op = tf.initialize_all_variables() 已过时
init_op = tf.global_variables_initializer()
```

变量的保存使用：**tf.train.Saver**

在神经网络中，Variable一般都是模型参数，在每轮训练时保存参数的更新，还可以将模型以参数的形式保存下来。

#### 1.4.1 保存变量

In [4]:
# 创建变量
v1 = tf.Variable([1.0, 2.0], name="v1")
v2 = tf.Variable([3.0, 3.0], name="v2")

# 初始化变量的op
init_op = tf.global_variables_initializer()

# 初始化saver的op
saver = tf.train.Saver()

# 注意，上面的两步都是op，真正执行也是在session中。

with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  # ...
  # 保存变量到硬盘
  save_path = saver.save(sess, "/tmp/model.ckpt")
  print "Model saved in file: ", save_path

Model saved in file:  model.ckpt


#### 1.4.2 加载变量

In [21]:
saver = tf.train.Saver()

with tf.Session() as sess:
  # 从硬盘加载变量
  saver.restore(sess, "/tmp/model.ckpt")
  print "Model restored."
  # Do some work with the model

Model restored.


### 1.5 Fetch 指定计算的结点

为了取回操作的输出内容, 可以在使用 Session 对象的 run() 调用执行图时, 传入一些tensor, 这些 tensor会帮助你取回结果。

run函数的第一个参数，传入fetch的list。

In [6]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)
intermed = tf.add(input2, input3)
mul = tf.mul(input1, intermed)

with tf.Session() as sess:
  result = sess.run([mul, intermed])
  print result

# 输出:
# [array([ 21.], dtype=float32), array([ 7.], dtype=float32)]

[21.0, 7.0]


### 1.6 Feed 图的输入

TensorFlow 还提供了 feed 机制, 该机制可以临时替代图中的任意操作中的 tensor 可以对图中任何操作提交补丁, 直接插入一个 tensor。

首先在构建图的时候要使用占位符：tf.placeholder。

然后在run函数里，指定feed_dict。

In [7]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.mul(input1, input2)

with tf.Session() as sess:
  print sess.run([output], feed_dict={input1:[7.], input2:[2.]})

# 输出:
# [array([ 14.], dtype=float32)]

[array([ 14.], dtype=float32)]
